In [1]:
import numpy as np
import pandas as pd
import re
import string

import nltk
from bangla_stemmer.stemmer import stemmer
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [29]:
def read_files(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        lines = []
        for line in file:
            lines.append(line)
        file.close()
    return lines

In [30]:
negative_lines = read_files(filename = "all_negative.txt")
positive_lines = read_files(filename = "all_positive.txt")


In [33]:
negative_lines[12]

' স্টুডেন্ট টারে ইচ্ছে করে কসায় একটা টাপ্পর মারি।\n'

In [31]:
def strip_lines(lines):
    modified_lines = []
    for i in range(len(lines)):
        temp = lines[i]
        temp = temp.lstrip().rstrip("\n")
        modified_lines.append(temp)
    return modified_lines

In [34]:
stripped_negative = strip_lines(negative_lines)
stripped_positive = strip_lines(positive_lines)

In [35]:
stripped_negative[23]

'মেহজাবিন এর ছাত্রী এটা? এই ছাগীটাকে প্রায় নাটকেই নেয় কেন এই পরিচালক?? এই আবাল ছেরি কি পরিচালকের কিছু হয়?? নাটকই দেখব না আর এই পরিচালকের। ধুর'

In [36]:
stripped_negative.pop(410)

'আমার চ্যানেলটা ঘুরে আসবেন আশা করি,♣♣♣♣♣♣♣দয়াকরে যাদের হার্ট দুর্বল তারা আসবেন না♦♦♦♦♦'

In [37]:
len(stripped_negative)
len(stripped_positive)

8500

In [38]:
def remove_punc(lines):
    punctuations = '''!()-[]।{};:'"\,<>./?@#$%^&*_~'''
    punch_remove_line = []
    # remove punctuation from the string
    for i in range(len(lines)):
        no_punct = ""
        for char in lines[i]:
            if char not in punctuations:
                no_punct = no_punct + char
        punch_remove_line.append(no_punct)
    return punch_remove_line

In [39]:
positive_lines = remove_punc(stripped_positive)
negative_lines = remove_punc(stripped_negative)

In [40]:
positive_lines[20]

'ভালো  লাগছে অসাধারণ হয়েছে নিসো ভাই'

In [41]:
def remove_english_words(lines):
    sentences = []
    for i in range(len(lines)):
        temp = lines[i]
        lst = temp.split()
        flag = 0
        for word in lst:
            x = re.search(r"^[a-zA-Z0-9]", word)
            if x:
                flag += 1
        if flag == 0:
            sentences.append(temp)
    return sentences

In [42]:
positive_lines = remove_english_words(positive_lines)
negative_lines = remove_english_words(negative_lines)

In [43]:
positive_lines[20]

'খুব ভাল লাগলো'

In [13]:
len(positive_lines)

7610

In [14]:
len(negative_lines)

3122

In [45]:
def create_dataset_csv(lines, filename, number):
    df = pd.DataFrame(lines, columns=["text"])
    if number == 0:
        label = np.zeros(len(lines))
        df['label'] = label
    elif number == 1:
        label = np.ones(len(lines))
        df['label'] = label
    df.to_csv(filename, index=False)

In [46]:
create_dataset_csv(positive_lines, "positive.csv", 1)
create_dataset_csv(negative_lines, "negative.csv", 0)

In [47]:
df_pos = pd.read_csv("positive.csv")
df_neg = pd.read_csv("negative.csv")

In [48]:
df_pos_new = df_pos[:3122]

,text,label
0,অসাধারণ নিশো বস্ আর অমি ভাইকেও,1.0
1,আমার দেখা বেস্ট নাটক,1.0
2,সত্যি অসাধারণ একটি রিলেশন,1.0
3,মজা পাইছি ভাষা গুলো কেমন লাগলো,1.0
4,খুব সুন্দর হয়েছে,1.0
...,...,...
3117,আরিয়ান ভাইয়ের নাটক গুলো অনেক ভালো আর নিশো ভাই ...,1.0
3118,ভালোবাসা সবাই যেনো তার মনের মানুষকে তার নিজের ...,1.0
3119,মেহজাবিনের ইনোসেন্ট চরিত্র টা অনেক ভাল লাগছে,1.0
3120,বলার ভাষা নাই অসাধারণ নিশুও মেহজাবিন,1.0


In [49]:
len(df_pos_new)

3122

In [50]:
len(df_neg)

3122

In [51]:
out = df_pos_new.append(df_neg)

In [52]:
out.to_csv("new_file.csv", index = False)

In [53]:
finaldf = pd.read_csv("new_file.csv")

In [54]:
np.random.seed(10)
finaldf = finaldf.sample(frac = 1) # shuffle
finaldf

,text,label
5000,বেগরাইন্ড মিউজিক এর জন্য কথা শুনা যাচ্ছে না তা...,0.0
2153,এত সুন্দর কেন এই নাটকগুলো,1.0
2878,অনেক দিন পর ভালো একটা নাটক দেখলাম,1.0
4858,বাজে নাটক বেহায়া ও ছেচরা নায়িকা রোল মেয়েদের কল...,0.0
655,ভাই নিশো অসাধারণ একটা নাটক কিন্তু অনেক কস্ট পেলাম,1.0
...,...,...
1180,ভাই এই নাটকটির শেষ করতে পারলে ভালো লাগতো যাই হ...,1.0
3441,সবচেয় বাজে নাটক গুলার মধ্যে একটা৷,0.0
1344,অসাধারন লাগছে,1.0
4623,সাবিলার কথা বলার ষ্টাইল ভালো লাগে নাচিটাগাংইয়া...,0.0


In [ ]:
# data preprocess and stuff

In [55]:
stop_words = []
with open('bangla_sw.txt','r') as file:    
    for line in file:      
        for word in line.split():         
            stop_words.append(word)
print(type(stop_words))
print(len(stop_words))

<class 'list'>
398


In [5]:
X = list()
Y = list()
for index, row in finaldf.iterrows():
    X.append(getattr(row, "text"))
    Y.append(getattr(row, "label"))

In [6]:
def process_senteence(sentence):
    stmr = stemmer.BanglaStemmer()
    
    tokenizer = WhitespaceTokenizer()
    tokens = tokenizer.tokenize(sentence)

    sentence_clean = []
    for word in tokens:
        if (word not in stop_words):
            stem_word = stmr.stem(word)  # stemming word
            sentence_clean.append(stem_word)

    return sentence_clean


def build_freqs(sentences, ys):
    yslist = np.squeeze(ys).tolist()
    freqs = {}
    for y, sentence in zip(yslist, sentences):
        for word in process_senteence(sentence):
            # print(word)
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
    return freqs

In [8]:
x_train,x_test,y_train,y_test= train_test_split(X,Y,test_size=0.2)

In [59]:
x_train[23]

'সেরা একটা নাটক উপহার দিয়েছেন যেনো একটা বলিউড মুভি দেখেছিনাটক  বস আফরান নিশো ভাই'

In [9]:
freqs = build_freqs(x_train, y_train)

print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
appl

applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
appl

applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
ap

applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
appli

applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied third rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied third rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied second rules..
applied

...

applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
appli

applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
app

applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
appl

applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
appl

applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applie

In [10]:
freqs

{('শেষের', 1.0): 25,
 ('দিখে', 1.0): 1,
 ('ভাল', 1.0): 226,
 ('লাগছিল', 1.0): 1,
 ('পুরো', 0.0): 19,
 ('গল্প', 0.0): 74,
 ('নেতিবাচক', 0.0): 1,
 ('উষ্কানিমৃলক', 0.0): 1,
 ('বাংলাভিশনে', 0.0): 1,
 ('এরকম', 0.0): 47,
 ('গাজাখোরী', 0.0): 1,
 ('নাটক', 0.0): 1682,
 ('চ্যানেলের', 0.0): 1,
 ('ভরসা', 0.0): 1,
 ('হারা', 0.0): 15,
 ('খারাপ', 0.0): 107,
 ('সুন্দর', 0.0): 23,
 ('মার্জিত', 0.0): 2,
 ('অশ্লীল', 0.0): 53,
 ('বঙ্গিমা', 0.0): 2,
 ('কথা', 0.0): 86,
 ('বার্তা', 0.0): 9,
 ('ছি', 0.0): 35,
 ('লজ্জ', 0.0): 1,
 ('মাথা', 0.0): 36,
 ('খাইলো', 0.0): 1,
 ('পরিচালকআরে', 0.0): 1,
 ('ভাই', 0.0): 130,
 ('বাংলাদেশ', 0.0): 21,
 ('আমিরিকা', 0.0): 1,
 ('নাএ', 0.0): 7,
 ('ডিরেক্টর', 0.0): 25,
 ('পরিচালক', 0.0): 118,
 ('জুতা', 0.0): 11,
 ('পিটা', 0.0): 3,
 ('ব্যান্ড', 0.0): 1,
 ('ফাজলামো', 0.0): 2,
 ('এত', 1.0): 45,
 ('ভালো', 1.0): 794,
 ('লাগলো', 1.0): 281,
 ('শেষ', 1.0): 81,
 ('যাবেনা', 1.0): 3,
 ('নিশো', 1.0): 313,
 ('ভাই', 1.0): 410,
 ('মানে', 1.0): 54,
 ('ভিন্ন', 1.0): 10,
 ('বাজে', 0.0): 205,
 ('এক'

In [11]:
def extract_features(sentence, freqs):
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_senteence(sentence)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1
    for word in word_l:
        x[0,1] += freqs.get((word, 1.0),0)
        x[0,2] += freqs.get((word, 0.0),0)
    assert(x.shape == (1, 3))
    return x

In [12]:
X = np.zeros((len(x_train), 3))
for i in range(len(x_train)):
    X[i, :]= extract_features(x_train[i], freqs)

applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
appl

applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied third rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied

applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
appli

applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..

applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
a

...

applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
app

applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied 

applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rule

applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
appli

applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
ap

In [13]:
from sklearn.linear_model import LogisticRegression

In [58]:
X[23]

array([1.000e+00, 3.526e+03, 2.639e+03])

In [14]:
logreg = LogisticRegression()
logreg.fit(X, y_train)
X_test = np.zeros((len(x_test), 3))
for i in range(len(x_test)):
    X_test[i, :]= extract_features(x_test[i], freqs)

applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
appl

applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
a

applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applie

applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied third rules..
applied second rules..
applied second rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applie

applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
appl

applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied third rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied third rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
appl

applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..


applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied first rules..
appl

applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
app

In [15]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.4f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.8439


In [16]:
# precision, recall, f1, accuracy
from sklearn import metrics
import numpy as np
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.83      0.87      0.85       631
         1.0       0.86      0.82      0.84       618

    accuracy                           0.84      1249
   macro avg       0.84      0.84      0.84      1249
weighted avg       0.84      0.84      0.84      1249



In [25]:
accurecy_score = metrics.accuracy_score(y_test,y_pred)
precion_score = metrics.precision_score(y_test,y_pred)
recall_score = metrics.recall_score(y_test,y_pred)
f1_score = metrics.f1_score(y_test,y_pred)



print("Accuracy: ", accurecy_score)
print("Precision: ", precion_score)
print("Recall: ", recall_score)
print("F1 Score: ",f1_score)

Accuracy:  0.8438751000800641
Precision:  0.8603066439522998
Recall:  0.8171521035598706
F1 Score:  0.8381742738589211


In [36]:
# Sentiment Analysis 
# 1. logistic regression (Done)
# 2. Naive Bayes (Done)
# 3. RNN (GRU) / gru+lstm (Done)
# 4. RNN (LSTM) (Done)
# 5. Attention Model
# 6. SVM

In [17]:
# naive Bayes
from sklearn.naive_bayes import MultinomialNB # Multinomial Naive Bayes
from sklearn.naive_bayes import GaussianNB # Gaussian Naive Bayes
from sklearn.naive_bayes import BernoulliNB # Bernoulli Naive Bayes
from sklearn.naive_bayes import ComplementNB # Complement Naive Bayes

from sklearn import metrics

In [24]:
# Multinomial Naive Bayes
MNB = MultinomialNB()
MNB.fit(X,y_train)
predicted = MNB.predict(X_test)
accurecy_score = metrics.accuracy_score(y_test,predicted)
precion_score = metrics.precision_score(y_test,predicted)
recall_score = metrics.recall_score(y_test,predicted)
f1_score = metrics.f1_score(y_test,predicted)


print(str('MNB Accurecy on Test Set: {:04.2f}'.format(accurecy_score*100)) + '%')
print("Accuracy: ", accurecy_score)
print("Precision: ", precion_score)
print("Recall: ", recall_score)
print("F1 Score: ",f1_score)


MNB Accurecy on Test Set: 82.79%
Accuracy:  0.8278622898318655
Precision:  0.8234349919743178
Recall:  0.8300970873786407
F1 Score:  0.8267526188557615


In [28]:
# Complement Naive Bayes
CNB = ComplementNB()
CNB.fit(X,y_train)
predicted_CNB = CNB.predict(X_test)
accurecy_score_CNB = metrics.accuracy_score(predicted_CNB,y_test)
print(str('CNB Accurecy on Test Set: {:04.2f}'.format(accurecy_score_CNB*100)) + '%')

CNB Accurecy on Test Set: 82.79%


In [26]:
# Gaussian Naive Bayes 
GNB = GaussianNB()
GNB.fit(X,y_train)
predicted_GNB = GNB.predict(X_test)
accurecy_score_GNB = metrics.accuracy_score(y_test, predicted_GNB)
precion_score_GNB = metrics.precision_score(y_test,predicted_GNB)
recall_score_GNB = metrics.recall_score(y_test,predicted_GNB)
f1_score_GNB = metrics.f1_score(y_test,predicted_GNB)

print(str('GNB Accurecy on Test Set: {:04.2f}'.format(accurecy_score_GNB*100)) + '%')

print("Accuracy: ", accurecy_score_GNB)
print("Precision: ", precion_score_GNB)
print("Recall: ", recall_score_GNB)
print("F1 Score: ",f1_score_GNB)

GNB Accurecy on Test Set: 71.02%
Accuracy:  0.7101681345076061
Precision:  0.8878787878787879
Recall:  0.4741100323624595
F1 Score:  0.6181434599156117


In [27]:
# Bernoulli Naive Bayes
BNB = BernoulliNB()
BNB.fit(X,y_train)
predicted_BNB = BNB.predict(X_test)
accurecy_score_BNB = metrics.accuracy_score(predicted_BNB,y_test)
print(str('BNB Accurecy on Test Set: {:04.2f}'.format(accurecy_score_BNB*100)) + '%')

BNB Accurecy on Test Set: 50.36%
